## 売上資料Ⅰ
## 月別損益計算書　作成

- 作成年度と作成月の入力を忘れずに！

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 作成年度を入力
nendo = 2021
# 作成月を入力（1～12を入力）
month = 4

# ファイル名指定
file_nm = "2021年度経費データ.csv"

In [ ]:
promotion_cd = [841]
travel_cd = [842001]
reception_cd = [843003, 844001]
kamoku_cd_list = [846,847,849,850,856,857,875,845001,848002,852001,855001,859001,859002,859003,869002]
sample_nm = "販促ｻﾝﾌﾟﾙ等無償出庫分"

In [ ]:
import numpy as np
import pandas as pd
import re

In [ ]:
def month_set(nendo, month):
    """年と月で6桁の年月にする
    """
    if month < 6:
        this_month = int(str(nendo+1) + "0" + str(month))
    else:
        if month > 9:
            this_month = int(str(nendo) + str(month))
        else:
            this_month = int(str(nendo) + "0" + str(month))
    
    return this_month

In [ ]:
def mark_num_change(mark):
    """
        半角￥マークつきの金額部分の数値のみ取り出す
              例）販売価格 \824,529 --> 824529
    """
    if len(mark) >= 2:
        if mark[-2] == ".":
            mark = mark[:-2]
    # 「\」と「,」と数字以外の文字を消去する
    after = re.sub(r"\D", "", mark)
    # マイナス対応
    if "-" in mark:
        after = int(after) * (-1)
    else:
        after = int(after)
    return after

In [ ]:
# 前年度を設定
zen_nendo = nendo - 1

# 当月を設定
this_month = month_set(nendo, month)

# 前年当月を設定
zen_month = month_set(zen_nendo, month)

# 当年の最終月を設定
end_month = month_set(nendo, 5)

# 12ヵ月分のlistを作成
list_12 = []
m = 6
y = month_set(nendo, m)
list_12.append(y)
for i in range(11):
    m += 1
    if m > 12:
        m = 1
    y = month_set(nendo, m)
    list_12.append(y)
print('当月：', this_month)
print('前年当月', zen_month)
print('最終月', end_month)
print(list_12)

当月： 202204
前年当月 202104
最終月 202205
[202106, 202107, 202108, 202109, 202110, 202111, 202112, 202201, 202202, 202203, 202204, 202205]


In [ ]:
# 当年度、前年度の棚卸月を設定
tanaorosi_nendo = []
tanaorosi_zen_nendo = []

tmp_month = int(str(nendo) + "0" + str(5))
tanaorosi_nendo.append(tmp_month)
tmp_month = int(str(nendo-1) + "0" + str(5))
tanaorosi_zen_nendo.append(tmp_month)

start_m = 6
tmp_month = month_set(nendo, start_m)

while tmp_month <= this_month:   # 当年度セット
    tanaorosi_nendo.append(tmp_month)
    if tmp_month == this_month:
        break
    
    start_m += 1
    if start_m > 12:
        start_m = 1
    tmp_month = month_set(nendo, start_m)

start_m = 6
tmp_month = month_set(zen_nendo, start_m)

while tmp_month <= zen_month:   # 前年度セット
    tanaorosi_zen_nendo.append(tmp_month)
    if tmp_month == zen_month:
        break
    
    start_m += 1
    if start_m > 12:
        start_m = 1
    tmp_month = month_set(zen_nendo, start_m)

# print
print('nendo= {}'.format(nendo))
print('tanaorosi_nendo= {}'.format(tanaorosi_nendo))
print('tanaorosi_zen_nendo= {}'.format(tanaorosi_zen_nendo))

nendo= 2021
tanaorosi_nendo= [202105, 202106, 202107, 202108, 202109, 202110, 202111, 202112, 202201, 202202, 202203, 202204]
tanaorosi_zen_nendo= [202005, 202006, 202007, 202008, 202009, 202010, 202011, 202012, 202101, 202102, 202103, 202104]


In [ ]:
target_col = ['年度','売上月','請求先CD','部門','大分類','小分類','伝票区分','品目','品名','規格','販売数量','販売単価','販売金額']
uriagedf_n = pd.read_csv("/content/drive/MyDrive/Purchased_products/csv/N_U_2020_2022.csv",usecols=target_col)

target_col = ['棚卸月','大分類CD','大分類名','品目','品名','規格','在庫数','在庫単価','在庫金額']
zaikodf_i = pd.read_csv("/content/drive/MyDrive/Purchased_products/csv/I_Z_2009_2022.csv",usecols=target_col)

target_col = ['年度','売上月','大分類','小分類','伝票区分','品目','品名','規格','販売数量','販売単価','販売金額','仕入金額']
uriagedf_i = pd.read_csv("/content/drive/MyDrive/Purchased_products/csv/I_U_2009_2022.csv",usecols=target_col)

target_col = ['伝票日付','借方','借方科目名','金額','摘要']
keihidf_i = pd.read_csv("/content/drive/MyDrive/Purchased_products/csv/" + file_nm, usecols=target_col, encoding="cp932")

In [ ]:
# uriagedf_n 整理
uriagedf_n = uriagedf_n[uriagedf_n['請求先CD'] != 'Plastic Jaw Frame(UP-13)']
uriagedf_n = uriagedf_n[uriagedf_n['請求先CD'] != '114_器材']
uriagedf_n = uriagedf_n[uriagedf_n['請求先CD'] != ' ']
uriagedf_n = uriagedf_n.fillna(0)
uriagedf_n["売上月"] = uriagedf_n["売上月"].astype(int)
uriagedf_n["品目"] = uriagedf_n["品目"].astype(int)
uriagedf_n["販売数量"] = uriagedf_n["販売数量"].astype(int)
uriagedf_n["販売単価"] = uriagedf_n["販売単価"].astype(int)
uriagedf_n["販売金額"] = uriagedf_n["販売金額"].astype(int)
uriagedf_n["年度"] = uriagedf_n["年度"].astype(int)
uriagedf_n["請求先CD"] = uriagedf_n["請求先CD"].astype(int)

In [ ]:
# uriagedf_i 整理
uriagedf_i = uriagedf_i.fillna(0)
uriagedf_i["売上月"] = uriagedf_i["売上月"].astype(int)
uriagedf_i["販売数量"] = uriagedf_i["販売数量"].astype(int)
uriagedf_i["販売単価"] = uriagedf_i["販売単価"].astype(int)
uriagedf_i["販売金額"] = uriagedf_i["販売金額"].astype(int)
uriagedf_i["年度"] = uriagedf_i["年度"].astype(int)

In [ ]:
# zaikodf_i 整理
zaikodf_i = zaikodf_i.fillna(0)
zaikodf_i['棚卸月'] = zaikodf_i['棚卸月'].astype(int)
zaikodf_i['在庫数'] = zaikodf_i['在庫数'].astype(int)
zaikodf_i['在庫単価'] = zaikodf_i['在庫単価'].astype(int)
zaikodf_i['在庫金額'] = zaikodf_i['在庫金額'].astype(int)

In [ ]:
# kihidfi 整理
keihidf_i["伝票月"] = (keihidf_i['伝票日付'][:4] + keihidf_i['伝票日付'][5:-1])

In [ ]:
keihidf_i.head()

In [ ]:
def date_henkan(date):
    point_cnt = 0
    henkan_list = ['00','01','02','03','04','05','06','07','08','09','10','11','12']
    for i in range(len(date)):
        if point_cnt == 2:
            mm = henkan_list[mm]
            yyyymm = int(str(yyyy) + mm)
            break

        if (point_cnt == 1) and (date[i] == "."):
            mm = int(date[first_point:i])
            point_cnt += 1

        if (point_cnt == 0) and (date[i] == "."):
            ry = int(date[1:i])
            yyyy = ry + 2018
            first_point = i + 1
            point_cnt += 1
    return yyyymm

In [ ]:
# yyyy/mm/dd -> yyyy/mm
ym_list = []
for i in range(len(keihidf_i)):
    date_keihi = date_henkan(keihidf_i.iloc[i]['伝票日付'])
    ym_list.append(date_keihi)
keihidf_i['伝票月'] = ym_list

# 金額を数字に
keihidf_i = keihidf_i.fillna(0)
kingaku_list = []
for i in range(len(keihidf_i)):
    kingaku_list.append(mark_num_change(str(keihidf_i.iloc[i]['金額'])))
keihidf_i['金額'] = kingaku_list

### データ作成

In [ ]:
# 仕入データ絞り込み
uriagedf_n = uriagedf_n[(uriagedf_n["売上月"] >= tanaorosi_zen_nendo[0]) & (uriagedf_n["売上月"] <= tanaorosi_nendo[-1])]
uriagedf_n = uriagedf_n[uriagedf_n["請求先CD"] == 10233]

# 月ごと、伝票区分ごとをdictへ
total_siire_dict = dict(uriagedf_n.groupby(["売上月"])["販売金額"].sum())

In [ ]:
uriagedf_n['売上月'].unique()

array([202005, 202006, 202007, 202008, 202011, 202009, 202010, 202012,
       202101, 202102, 202105, 202104, 202103, 202106, 202107, 202108,
       202201, 202112, 202111, 202110, 202109, 202202, 202204, 202203])

In [ ]:
# 売上データ絞り込み
uriagedf_i = uriagedf_i[(uriagedf_i["売上月"] >=tanaorosi_zen_nendo[0]) & (uriagedf_i["売上月"] <= tanaorosi_nendo[-1])]

# 月ごと、伝票区分ごとをdictへ
total_uriage_dict = dict(uriagedf_i.groupby(["売上月", "伝票区分"])["販売金額"].sum())

In [ ]:
uriagedf_i["売上月"].unique()

array([202005, 202006, 202007, 202008, 202009, 202010, 202011, 202012,
       202101, 202102, 202103, 202104, 202105, 202106, 202107, 202108,
       202109, 202110, 202111, 202112, 202201, 202202, 202203, 202204])

In [ ]:
# 在庫データの絞り込み
zaikodf_i[zaikodf_i["棚卸月"].isin(tanaorosi_nendo+tanaorosi_zen_nendo)]

# 棚卸月毎に在庫金額を集計
total_zaiko_dict = dict(zaikodf_i.groupby('棚卸月')['在庫金額'].sum())

In [ ]:
keihidf_i.head(-5)

In [ ]:
# 経理データ集計
total_keihi_1_dict = {}
total_keihi_2_dict = {}
total_keihi_3_dict = {}
total_keihi_4_dict = {}
total_keihi_5_dict = {}
total_keihi_6_dict = {}
total_keihi_7_dict = {}
for i in range(len(keihidf_i)):
    if keihidf_i.iloc[i]['借方'] in promotion_cd:
        if keihidf_i.iloc[i]['摘要'][0:13] == sample_nm:
            if keihidf_i.iloc[i]['伝票月'] not in total_keihi_2_dict:
                total_keihi_2_dict[keihidf_i.iloc[i]['伝票月']] = keihidf_i.iloc[i]['金額']
            else:
                total_keihi_2_dict[keihidf_i.iloc[i]['伝票月']] += keihidf_i.iloc[i]['金額']
        else:
            if keihidf_i.iloc[i]['伝票月'] not in total_keihi_1_dict:
                total_keihi_1_dict[keihidf_i.iloc[i]['伝票月']] = keihidf_i.iloc[i]['金額']
            else:
                total_keihi_1_dict[keihidf_i.iloc[i]['伝票月']] += keihidf_i.iloc[i]['金額']

    if keihidf_i.iloc[i]['借方'] in reception_cd:
        if keihidf_i.iloc[i]['伝票月'] not in total_keihi_5_dict:
            total_keihi_5_dict[keihidf_i.iloc[i]['伝票月']] = keihidf_i.iloc[i]['金額']
        else:
            total_keihi_5_dict[keihidf_i.iloc[i]['伝票月']] += keihidf_i.iloc[i]['金額']

    if keihidf_i.iloc[i]['借方'] in travel_cd:
        if keihidf_i.iloc[i]['伝票月'] not in total_keihi_6_dict:
            total_keihi_6_dict[keihidf_i.iloc[i]['伝票月']] = keihidf_i.iloc[i]['金額']
        else:
            total_keihi_6_dict[keihidf_i.iloc[i]['伝票月']] += keihidf_i.iloc[i]['金額']

    if keihidf_i.iloc[i]['借方'] in kamoku_cd_list:
        if keihidf_i.iloc[i]['伝票月'] not in total_keihi_7_dict:
            total_keihi_7_dict[keihidf_i.iloc[i]['伝票月']] = keihidf_i.iloc[i]['金額']
        else:
            total_keihi_7_dict[keihidf_i.iloc[i]['伝票月']] += keihidf_i.iloc[i]['金額']

In [ ]:
print(total_keihi_1_dict)
print(total_keihi_2_dict)
print(total_keihi_3_dict)
print(total_keihi_4_dict)
print(total_keihi_5_dict)
print(total_keihi_6_dict)
print(total_keihi_7_dict)

{202106: 515756, 202107: 176759, 202108: 857852, 202109: 180191, 202110: 2027093, 202111: 367437, 202112: 1002371, 202201: 118346, 202202: 281423, 202203: 1151592, 202204: 1217527}
{}
{}
{}
{202107: 46524, 202108: 4677, 202109: 8251, 202110: 6424, 202111: 118327, 202112: 6674, 202201: 48622, 202202: 19546, 202203: 7387}
{202106: 151781, 202107: 498455, 202108: 1020015, 202109: 512022, 202110: 585304, 202111: 1253186, 202112: 616753, 202201: 892046, 202202: 910948, 202203: 1334208, 202204: 686070}
{202106: 2752295, 202107: 3142673, 202108: 3149326, 202109: 3262092, 202110: 3384806, 202111: 3014499, 202112: 3832331, 202201: 3641347, 202202: 3489234, 202203: 3640755, 202204: 3896202}


In [ ]:
# 月別集計用dict作成
syukei_A_dict = {}
syukei_B_dict = {}
syukei_CE_dict = {}
syukei_F_dict = {}
syukei_GK_dict = {}
for i in range(len(list_12)):
    syukei_A_dict[list_12[i]] = [0,0,0,0,0,0,0,0,0]
    syukei_B_dict[list_12[i]] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    syukei_CE_dict[list_12[i]] = [0,0,0,0,0,0]
    syukei_F_dict[list_12[i]] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    syukei_GK_dict[list_12[i]] = [0,0,0,0,0,0,0,0,0,0,0,0]

In [ ]:
def zennen_keisan(nen_month):
    nen_month_str = str(nen_month)
    nen_int = int(nen_month_str[:4])
    nen_month_str = str(nen_int - 1) + nen_month_str[4:]

    return int(nen_month_str)

In [ ]:
# 売上セット
for i in syukei_A_dict.keys():
    k = zennen_keisan(i)
    if (i, "売上") in total_uriage_dict:
        syukei_A_dict[i][0] = total_uriage_dict[k, '売上'] # 0:前年売上
        syukei_A_dict[i][2] = total_uriage_dict[i, '売上'] # 2:売上
        syukei_A_dict[i][3] = (total_uriage_dict[k, '値引'] + total_uriage_dict[k, '返品'] \
                                                        + total_uriage_dict[k, '見本']) * (-1) # 3:前年値引
        syukei_A_dict[i][5] = (total_uriage_dict[i, '値引'] + total_uriage_dict[i, '返品'] \
                                                        + total_uriage_dict[i, '見本']) * (-1) # 3:値引
        syukei_A_dict[i][6] = syukei_A_dict[i][0] - syukei_A_dict[i][3] # 6:前年純売り
        syukei_A_dict[i][8] = syukei_A_dict[i][2] - syukei_A_dict[i][5] # 8:純売り

In [ ]:
# 仕入セット
for i in syukei_B_dict.keys():
    k = zennen_keisan(i)
    if i in total_siire_dict:
        syukei_B_dict[i][2] = total_siire_dict[k] # 2:前年仕入
        syukei_B_dict[i][4] = total_siire_dict[i] # 4:仕入

In [ ]:
def zengetu_keisan(nen_month):
    nen_month_str = str(nen_month)
    month_int = int(nen_month_str[-2:]) - 1
    if month_int == 0:
        month_int = 12
        nen_int = int(nen_month_str[:4]) - 1
    else:
        nen_int = int(nen_month_str[:4])

    if month_int < 10:
        nen_month_str = str(nen_int) + "0" + str(month_int)
    else:
        nen_month_str = str(nen_int) + str(month_int)

    return int(nen_month_str)

In [ ]:
# 在庫セット
for i in syukei_B_dict.keys():
    if i > this_month:
        break
    k = zennen_keisan(i)  # k:前年当月
    m = zengetu_keisan(i) # m:前月
    t = zengetu_keisan(k) # t:前年前月
    syukei_B_dict[i][0] = total_zaiko_dict[t] # 0:前年期首在庫
    syukei_B_dict[i][1] = total_zaiko_dict[m] # 0:期首在庫
    syukei_B_dict[i][8] = total_zaiko_dict[k] # 0:前年期末在庫
    syukei_B_dict[i][9] = total_zaiko_dict[i] # 0:期末在庫
    syukei_B_dict[i][10] = syukei_B_dict[i][0] + syukei_B_dict[i][2] \
                                               - syukei_B_dict[i][5] - syukei_B_dict[i][8]
    syukei_B_dict[i][11] = syukei_B_dict[i][1] + syukei_B_dict[i][4] \
                                               - syukei_B_dict[i][7] - syukei_B_dict[i][9]

In [ ]:
# 変動費率、付加価値額、付加価値額率
for i in syukei_CE_dict.keys():
    if syukei_A_dict[i][6] == 0:
        syukei_CE_dict[i][0] = 0
    else:
        syukei_CE_dict[i][0] = syukei_B_dict[i][10] / syukei_A_dict[i][6] #0:変動費率（前年）百分率にはしていない

    if syukei_A_dict[i][8] == 0:
        syukei_CE_dict[i][1] = 0
    else:
        syukei_CE_dict[i][1] = syukei_B_dict[i][11] / syukei_A_dict[i][8] #1:変動費率　百分率にはしていない

    syukei_CE_dict[i][2] = syukei_A_dict[i][6] - syukei_B_dict[i][10] #2:付加価値額（前年）
    syukei_CE_dict[i][3] = syukei_A_dict[i][8] - syukei_B_dict[i][11] #3:付加価値額

    if syukei_A_dict[i][6] == 0:
        syukei_CE_dict[i][4] = 0
    else:
        syukei_CE_dict[i][4] = syukei_CE_dict[i][2] / syukei_A_dict[i][6] #4:付加価値額率（前年）百分率にはしていない

    if syukei_A_dict[i][8] == 0:
        syukei_CE_dict[i][5] = 0
    else:
        syukei_CE_dict[i][5] = syukei_CE_dict[i][3] / syukei_A_dict[i][8] #5:付加価値額率　百分率にはしていない

In [ ]:
# 販売費セット
for i in syukei_F_dict.keys():
    gokei_num = 0
    if i in total_keihi_1_dict:
        syukei_F_dict[i][2] = total_keihi_1_dict[i]
        gokei_num += total_keihi_1_dict[i]
    if i in total_keihi_2_dict:
        syukei_F_dict[i][5] = total_keihi_2_dict[i]
        gokei_num += total_keihi_2_dict[i]
    if i in total_keihi_3_dict:
        syukei_F_dict[i][8] = total_keihi_3_dict[i]
        gokei_num += total_keihi_3_dict[i]
    if i in total_keihi_4_dict:
        syukei_F_dict[i][11] = total_keihi_4_dict[i]
        gokei_num += total_keihi_4_dict[i]
    if i in total_keihi_5_dict:
        syukei_F_dict[i][14] = total_keihi_5_dict[i]
        gokei_num += total_keihi_5_dict[i]
    if i in total_keihi_6_dict:
        syukei_F_dict[i][17] = total_keihi_6_dict[i]
        gokei_num += total_keihi_6_dict[i]
    if i in total_keihi_7_dict:
        syukei_F_dict[i][20] = total_keihi_7_dict[i]
        gokei_num += total_keihi_7_dict[i]

    syukei_F_dict[i][23] = gokei_num

In [ ]:
# 貢献利益、固定費、営業利益、営業利益率、特別損益
for i in syukei_GK_dict.keys():
    syukei_GK_dict[i][0] = syukei_CE_dict[i][2] - syukei_F_dict[i][21] # 貢献利益（前年）
    syukei_GK_dict[i][1] = syukei_CE_dict[i][3] - syukei_F_dict[i][23] # 貢献利益
    syukei_GK_dict[i][2] = 0 # 人件費（前年）
    syukei_GK_dict[i][4] = 0 # 人件費
    syukei_GK_dict[i][5] = syukei_GK_dict[i][0] - syukei_GK_dict[i][2] # 営業利益（前年）
    syukei_GK_dict[i][6] = syukei_GK_dict[i][1] - syukei_GK_dict[i][4] # 営業利益
    if syukei_A_dict[i][6] == 0:
        syukei_GK_dict[i][7] = 0
    else:
        syukei_GK_dict[i][7] = syukei_GK_dict[i][5] / syukei_A_dict[i][6] # 営業利益率（前年）
    if syukei_A_dict[i][8] == 0:
        syukei_GK_dict[i][8] = 0
    else:
        syukei_GK_dict[i][8] = syukei_GK_dict[i][6] / syukei_A_dict[i][8] # 営業利益率（前年）
    syukei_GK_dict[i][9] = 0 # 特別損失（前年）
    syukei_GK_dict[i][11] = 0 # 特別損失

In [ ]:
# syukei_x_dictの結合
syukei_dict = {}
for i in syukei_A_dict.keys():
    syukei_dict[i] = syukei_A_dict[i] + syukei_B_dict[i] + syukei_CE_dict[i] + syukei_F_dict[i] + syukei_GK_dict[i]

# Dataframe化
df = pd.DataFrame(syukei_dict, index=None)

In [ ]:
# Excelデータ作成
df.to_excel("/content/drive/MyDrive/Purchased_products/excel/営業資料Ⅰ.xlsx", sheet_name='月別損益計算')